In [4]:
from openai import AzureOpenAI
from datasets import load_dataset
from data_handling import concat_item_metadata, load_data
import os
import numpy as np
import multiprocessing
N_CPUS = multiprocessing.cpu_count()

def concat_item_metadata(dp):
    meta = ''
    flag = False
    if dp['title'] is not None:
        meta += dp['title']
        flag = True
    if len(dp['features']) > 0:
        if flag:
            meta += ' '
        meta += ' '.join(dp['features'])
        flag = True
    if len(dp['description']) > 0:
        if flag:
            meta += ' '
        meta += ' '.join(dp['description'])
    dp['cleaned_metadata'] = meta \
        .replace('\t', ' ') \
        .replace('\n', ' ') \
        .replace('\r', '') \
        .strip()
    return dp


reviews = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Books", trust_remote_code=True)
reviews = reviews['full']
reviews = reviews.sort('helpful_vote', reverse=True).take(5000)

item_meta = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Books", trust_remote_code=True)
item_meta = item_meta['full']
item_meta = item_meta.map(concat_item_metadata, num_proc=N_CPUS)

# https://github.com/hyp1231/AmazonReviews2023/blob/main/blair/sample_pretraining_data.py#L125


Loading dataset shards:   0%|          | 0/33 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/28 [00:00<?, ?it/s]

Map (num_proc=8):   0%|          | 0/4448181 [00:00<?, ? examples/s]

In [3]:
list(reviews.shuffle(seed=42).take(3).select_columns(['parent_asin', 'title', 'helpful_vote', 'text']))

[{'parent_asin': '1538714620',
  'title': 'A Different Approach to Healthy Diet',
  'helpful_vote': 623,
  'text': 'There are a large number of diet books around, espousing a variety of “healthy” lifestyles, and often these are unsustainable over the long term either for financial or health reasons. Many look at such factors as calories, fat, carbohydrates and more without really considering the foods themselves beyond that. This book isn’t a diet book, per se; instead, it looks at the individual foods and explains how the body uses them to heal and regenerate itself.<br /><br />To cut to the chase on one important point: the focus is very much on natural, unprocessed foods like fruits, vegetables, nuts and fish. There is a definite lack of coverage for land-based meats (beef, pork, lamb) though there is discussion of various types of seafood such as tuna, anchovies, clams and more, and chicken/turkey are also covered.<br /><br />This is not a formal diet book; rather, it is more of a 

In [6]:
all_meta = {}
candidate_asin = reviews.select_columns(["parent_asin"]).to_dict()["parent_asin"]
# efficiently filter item_meta to only include rows where parent_asin is in candidate_asin
item_meta_filtered = item_meta.filter(
    (lambda x: np.isin(x["parent_asin"], candidate_asin)),
    batched=True,
    num_proc=N_CPUS,
)
for row in item_meta_filtered:
    all_meta[row['parent_asin']] = row['cleaned_metadata']


Filter (num_proc=8):   0%|          | 0/4448181 [00:00<?, ? examples/s]

In [12]:
concat_item_metadata(all_meta[reviews[0]["parent_asin"]])['cleaned_metadata']

'What Happened A TIME BEST BOOK OF THE YEAR AND NEW YORK TIMES NOTABLE BOOK “In the past, for reasons I try to explain, I’ve often felt I had to be careful in public, like I was up on a wire without a net. Now I’m letting my guard down.” —Hillary Rodham Clinton, from the introduction of What Happened For the first time, Hillary Rodham Clinton reveals what she was thinking and feeling during one of the most controversial and unpredictable presidential elections in history. Now free from the constraints of running, Hillary takes you inside the intense personal experience of becoming the first woman nominated for president by a major party in an election marked by rage, sexism, exhilarating highs and infuriating lows, stranger-than-fiction twists, Russian interference, and an opponent who broke all the rules. This is her most personal memoir yet. In these pages, she describes what it was like to run against Donald Trump, the mistakes she made, how she has coped with a shocking and devasta

In [15]:
reviews = reviews.add_column("item_description", [all_meta[review['parent_asin']] for review in reviews])

ValueError: The table can't have duplicated columns but columns ['item_description'] are duplicated.

In [16]:
# few-shot prompt openai to review based on
def example_prompts(reviews):
    """
    Generate a list of few-shot prompts based on the reviews. 
    Assumes that the reviews have already been cleaned and have an 'item_description' column.
    """
    prompts = []
    for review in reviews:
        prompts.extend(
            [
                {"role": "user", "content": f"Item Description: {review['item_description']}"},
                {"role": "assistant", "content": f"Title: {review['title']}\nReview: {review['text']}"},
            ]
        )
    return prompts


def get_client():
    client = AzureOpenAI(
        azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
        api_version="2024-08-01-preview",
        api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    )
    return client


client = get_client()




In [20]:
seed = 42           
system_prompt = {
    "role": "system",
    "content": "You are a helpful assistant that can review books based on the item metadata.",
}
user_prompt = {
    "role": "user",
    "content": item_meta.shuffle(seed=seed).take(1)['cleaned_metadata'][0]
}
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        system_prompt,
        *example_prompts(reviews.shuffle(seed=seed).take(3)),
        user_prompt,
    ],
)

In [28]:
print(f"{'#'*10}\ngiven:\n{user_prompt['content']}")
print(f"{'#'*10}\nresponse:\n{response.choices[0].message.content}")

##########
given:
Outcast A breathtaking tale of love in the West--by the author of Autumnfire. Olivia Baron scandalizes the prudish matrons and rough cowboys of Elkhorn by practicing medicine. When a diptheria epidemic breaks out, she can finally prove her mettle to the townspeople--and to the handsome widower Gabe Danaker.
##########
response:
Title: A Heartfelt Western Romance
Review: "Outcast" is a captivating tale that beautifully intertwines love and resilience in the rugged backdrop of the American West. The story follows Olivia Baron, a pioneering woman who defies the traditional roles of her time by establishing herself as a doctor in the conservative town of Elkhorn. Her determination to practice medicine not only challenges societal norms but also sets the stage for her to prove her worth when a diphtheria epidemic strikes the community.

The narrative expertly showcases Olivia's character as she navigates the skepticism and prejudice of both the local matrons and the rough-